In [1]:
# Scraping
from bs4 import BeautifulSoup
import requests

import json
import pandas as pd

import pprint

from urllib.request import Request, urlopen

from time import sleep
from random import randint

In [2]:
# Progress counter/tracker
from contextlib import contextmanager
from timeit import default_timer
import sys
from datetime import timedelta

# Define a timer w/ decoractor to use with the loop
@contextmanager
def elapsed_timer():
    start=default_timer()
    elapser = lambda: default_timer() - start
    yield lambda: elapser()
    end = default_timer()
    elapser = lambda: end-start

# Testing w/ 2 radiologists

In [3]:
radiologists = ["Richard Duszak", "Andrew Rosenkrantz"]

In [48]:
prefix = 'https://www.doximity.com/pub/'
user_agent = {'User-agent' : 'Mozilla/5.0'}
input_name = radiologists[0].replace(' ', '-').lower()
degree = "-md"
url = prefix+input_name+degree
page = requests.get(url, headers=user_agent)
soup = BeautifulSoup(page.text, "html.parser")

#####################
full_name = soup.find("span", class_="user-full-name").text
first_name = soup.find("span", class_="user-name-first").text
last_name = soup.find("span", class_="user-name-last").text

specialty = soup.find("a", class_="profile-head-subtitle").text
city_state = soup.find("span", class_="profile-head-subtitle").text
subspecialty = soup.find("p", class_="user-subspecialty").text
job_title = soup.find("p", class_="user-job-title").text

st_address = soup.find("span", class_="black profile-contact-labels-wrap").text
PRE_phone_num = soup.find("div", class_="profile-contact-labels-wrap")
phone_num = PRE_phone_num.find("span", class_="black").text

ed_trn_elements = soup.find("ul", class_="profile-sectioned-list training").find_all("div", class_="profile-section-wrapper-text")
for element in ed_trn_elements:
    institution = element.find("span", class_="black").text
    education = element.find("span", class_="br").text
    print(institution)
    print(education)
    print()

########################
# print(full_name)
# print(first_name)
# print(last_name)
# print(specialty)
# print(city_state)
# print(subspecialty)
# print(job_title)
# print(st_address)
# print(phone_num)

University of Pennsylvania Health System
Fellowship, Interventional Radiology - Independent, 1994 - 1995

Duke University Hospital
Residency, Radiology-Diagnostic, 1990 - 1994

York Hospital
Internship, Transitional Year, 1989 - 1990

Pennsylvania State University College of Medicine
Class of 1989

La Salle University
B.A., Biology, Maxima Cum Laude, General University Honors, 1981 - 1985

[<div class="profile-section-wrapper-text"><span class="black" itemprop="name">University of Pennsylvania Health System</span><span class="br">Fellowship, Interventional Radiology - Independent, 1994 - 1995</span></div>, <div class="profile-section-wrapper-text"><span class="black" itemprop="name">Duke University Hospital</span><span class="br">Residency, Radiology-Diagnostic, 1990 - 1994</span></div>, <div class="profile-section-wrapper-text"><span class="black" itemprop="name">York Hospital</span><span class="br">Internship, Transitional Year, 1989 - 1990</span></div>, <div class="profile-section-w

In [6]:
for rad in radiologists:
    print(rad.replace(' ', '-').lower())

richard-duszak
andrew-rosenkrantz


In [58]:
# Initialize request details and lists to dump data dicts
prefix = 'https://www.doximity.com/pub/'
user_agent = {'User-agent' : 'Mozilla/5.0'}
verif_data = []
ed_trn_data = []

# Loop through each radiologist and scrape data of interest
with elapsed_timer() as elapsed:
    n_iter = 0
    
    for rad in radiologists:                                 
        input_name = rad.replace(' ', '-').lower()
        degree = "-md"
        url = prefix+input_name+degree
        page = requests.get(url, headers=user_agent)
        soup = BeautifulSoup(page.text, "html.parser")

        # Verification data
        verif_details = {}
        verif_details['input_name'] = input_name  #ID
        verif_details['url'] = url

        # Parse out exact text we want & add data to the dictionary
        try:
            verif_details['full_name'] = soup.find("span", class_="user-full-name").text
        except:
            pass

        try:
            verif_details['first_name'] = soup.find("span", class_="user-name-first").text
        except:
            pass

        try:
            verif_details['last_name'] = soup.find("span", class_="user-name-last").text
        except:
            pass

        try:
            verif_details['specialty'] = soup.find("a", class_="profile-head-subtitle").text
        except:
            pass

        try:
            verif_details['city_state'] = soup.find("span", class_="profile-head-subtitle").text
        except:
            pass

        try:
            verif_details['subspecialty'] = soup.find("p", class_="user-subspecialty").text
        except:
            pass

        try:
            verif_details['job_title'] = soup.find("p", class_="user-job-title").text
        except:
            pass

        try:
            verif_details['st_address'] = soup.find("span", class_="black profile-contact-labels-wrap").text
        except:
            pass

        try:
            verif_details['phone_num'] = soup.find("div", class_="profile-contact-labels-wrap").find("span", class_="black").text
        except:
            pass
        
        # Append the dictionaries to the list
        verif_data.append(verif_details)

        # Education and training data
        ed_trn_elements = soup.find("ul", class_="profile-sectioned-list training").find_all("div", class_="profile-section-wrapper-text")
        for element in ed_trn_elements:
            ed_trn_details = {}
            ed_trn_details['input_name'] = input_name  #ID
            
            # Parse out exact text we want & add data to the dictionary
            try:
                ed_trn_details['institution'] = element.find("span", class_="black").text
            except:
                pass

            try:
                ed_trn_details['education'] = element.find("span", class_="br").text
            except:
                pass
            
            # Append the dictionaries to the list
            ed_trn_data.append(ed_trn_details)
        
        # Counter and timer for progress checks  
        n_iter += 1
        if n_iter % 100 == 0:
            print("Iteration (radiologists): "+str(n_iter)+" ----- Time Elapsed: "+str(timedelta(seconds=round(elapsed()))))
            sys.stdout.flush()

            # Save intermediate output incase of crash or timeout
            filename1 = "datadump/doximity dump/saved_ed_trn_list.txt"
            with open(filename1, 'w') as f:
                for item in ed_trn_data:
                    f.write(f'{item}\n')
                    sys.stdout.flush()
            filename2 = "datadump/doximity dump/saved_verif_list.txt"
            with open(filename2, 'w') as f:
                for item in verif_data:
                    f.write(f'{item}\n')
                    sys.stdout.flush()

        # Iteration delay
        sleep(randint(1,2))

In [55]:
ed_trn_data
# verif_data

[{'input_name': 'richard-duszak',
  'institution': 'University of Pennsylvania Health System',
  'education': 'Fellowship, Interventional Radiology - Independent, 1994 - 1995'},
 {'input_name': 'richard-duszak',
  'institution': 'Duke University Hospital',
  'education': 'Residency, Radiology-Diagnostic, 1990 - 1994'},
 {'input_name': 'richard-duszak',
  'institution': 'York Hospital',
  'education': 'Internship, Transitional Year, 1989 - 1990'},
 {'input_name': 'richard-duszak',
  'institution': 'Pennsylvania State University College of Medicine',
  'education': 'Class of 1989'},
 {'input_name': 'richard-duszak',
  'institution': 'La Salle University',
  'education': 'B.A., Biology, Maxima Cum Laude, General University Honors, 1981 - 1985'},
 {'input_name': 'andrew-rosenkrantz',
  'institution': 'University of Maryland Program',
  'education': 'Residency, Radiology-Diagnostic, 2004 - 2008'},
 {'input_name': 'andrew-rosenkrantz',
  'institution': 'NYU Long Island School of Medicine',
 